In [1]:
#import library

import numpy as np                                             #to deal with the data
from sklearn.preprocessing import MinMaxScaler                 #to rescale data
from sklearn.model_selection import train_test_split           #to split train/set data
import copy
import keras                                                   #deep learning package
from keras.models import Sequential                            
from keras.layers import Conv2D, BatchNormalization, Flatten, Dense, Reshape, Activation

Using TensorFlow backend.


In [58]:
class Sudokusolver():
    
    def __init__(self, t_size):
        self.t_size = t_size
        
    def data_norm(self, data):
        scaler = MinMaxScaler(feature_range = (-0.5, 0.5))
        scaled_data = scaler.fit_transform(data)
        return scaled_data
    
    def data_denorm(self, sdata):
        return scaler.inverse_transform(sdata) 
    
    def read_data(self):
        quizzes = np.zeros((1000000, 81), np.int32)
        solutions = np.zeros((100000, 81), np.int32)
        for i, line in enumerate(open('sudoku.csv', 'r').read().splitlines()[1:]):
            quiz, solution = line.split(",")
            for j, q_s in enumerate(zip(quiz, solution)):
                q, s = q_s
                quizzes[i, j] = q
                solutions[i, j] = s
        quizzes = quizzes.reshape((-1, 9, 9, 1))
        solutions = solutions.reshape((-1, 81, 1))
        quizzes = self.data_norm(quizzes)
        solutions = self.data_norm(solutions)
        X_train, X_test, y_train, y_test = train_test_split(quizzes, solutions, test_size = self.t_size, random_state = 42)
        return X_train, X_test, y_train, y_test
    
    def train(self, batch_size, ep, save_model = False):
        
        model = Sequential()
        
        model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same', input_shape=(9,9,1)))
        model.add(BatchNormalization())
        model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same'))
        model.add(BatchNormalization())
        model.add(Conv2D(128, kernel_size=(1,1), activation='relu', padding='same'))

        model.add(Flatten())
        model.add(Dense(81 * 9))
        model.add(Reshape((-1, 9)))
        model.add(Activation('softmax'))

        model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
        model.fit(x_train, y_train, batch_size=batch_size, epochs=ep)
        
        return model
        
    def human_sudoku(self, sample):
        
        feat = copy.copy(sample)
        
        while(1):
            
            out = self.train.predict(feat.reshape((1, 9, 9, 1)))
            out = out.squeeze()
            
            pred = np.argmax(out, axis = 1).reshape((9, 9)) + 1
            prob = np.around(np.max(out, axis = 1).reshape((9, 9)), 2)
            
            feat = self.data_denorm(feat).reshape((9,9))
            mask = (feat==0)
     
            if(mask.sum()==0):
                break
            
            prob_new = prob*mask
    
            ind = np.argmax(prob_new)
            x, y = (ind//9), (ind%9)

            val = pred[x][y]
            feat[x][y] = val
            feat = self.data_norm(feat)
    
        return pred
    
    def solve_sudoku(self, game):
    
        game = game.replace('\n', '')
        game = game.replace(' ', '')
        game = np.array([int(j) for j in game]).reshape((9,9,1))
        game = self.data_norm(game)
        game = self.human_sudoku(game)
        return game

In [54]:
Sudokusolver = Sudokusolver(0.2)

In [55]:
model = Sudokusolver.train(64, 2)

Epoch 1/2
800000/800000 [==============================] - 1952s 2ms/step - loss: 0.4691
Epoch 2/2
800000/800000 [==============================] - 1960s 2ms/step - loss: 0.3609


In [57]:
model.save('sudoku.model')

In [73]:
game = '''
          0 8 0 0 3 2 0 0 1
          7 0 3 0 8 0 0 0 2
          5 0 0 0 0 7 0 3 0
          0 5 0 0 0 1 9 7 0
          6 0 0 7 0 9 0 0 8
          0 4 7 2 0 0 0 5 0
          0 2 0 6 0 0 0 0 9
          8 0 0 0 9 0 3 0 5
          3 0 0 8 2 0 0 1 0
      '''

In [66]:
def norm(a):
    
    return (a/9)-.5

def denorm(a):
    
    return (a+.5)*9

In [67]:
def inference_sudoku(sample):
    
    '''
        This function solve the sudoku by filling blank positions one by one.
    '''
    
    feat = copy.copy(sample)
    
    while(1):
    
        out = model.predict(feat.reshape((1,9,9,1)))  
        out = out.squeeze()

        pred = np.argmax(out, axis=1).reshape((9,9))+1     #index of the maximum entry in each column + 1 -> reshape
        prob = np.around(np.max(out, axis=1).reshape((9,9)), 2) 
        
        feat = denorm(feat).reshape((9,9))
        mask = (feat==0)
     
        if(mask.sum()==0):
            break
            
        prob_new = prob*mask
    
        ind = np.argmax(prob_new)
        x, y = (ind//9), (ind%9)

        val = pred[x][y]
        feat[x][y] = val
        feat = norm(feat)
    
    return pred


In [68]:
def solve_sudoku(game):
    
    game = game.replace('\n', '')
    game = game.replace(' ', '')
    game = np.array([int(j) for j in game]).reshape((9,9,1))
    game = norm(game)
    game = inference_sudoku(game)
    return game

In [97]:
pred

array([[4, 8, 4, 9, 3, 2, 7, 9, 1],
       [7, 1, 3, 9, 8, 6, 5, 9, 2],
       [5, 6, 2, 1, 1, 7, 8, 3, 4],
       [2, 5, 8, 4, 6, 1, 9, 7, 3],
       [6, 1, 2, 7, 5, 9, 1, 4, 8],
       [9, 4, 7, 2, 6, 6, 1, 5, 3],
       [4, 2, 5, 6, 7, 3, 8, 8, 9],
       [8, 6, 6, 1, 9, 4, 3, 2, 5],
       [3, 6, 5, 8, 2, 5, 6, 1, 7]], dtype=int64)

In [106]:
feat = denorm(feat).reshape((9,9))

In [113]:
mask

array([[ True, False,  True,  True, False, False,  True,  True, False],
       [False,  True, False,  True, False,  True,  True,  True, False],
       [False,  True,  True,  True,  True, False,  True, False,  True],
       [ True, False,  True,  True,  True, False, False, False,  True],
       [False,  True,  True, False,  True, False,  True,  True, False],
       [ True, False, False, False,  True,  True,  True, False,  True],
       [ True, False,  True, False,  True,  True,  True,  True, False],
       [False,  True,  True,  True, False,  True, False,  True, False],
       [False,  True,  True, False, False,  True,  True, False,  True]])

In [111]:
prob_new = prob*mask

In [112]:
prob_new

array([[0.96, 0.  , 0.68, 0.58, 0.  , 0.  , 0.76, 0.69, 0.  ],
       [0.  , 0.57, 0.  , 0.49, 0.  , 0.5 , 0.93, 0.76, 0.  ],
       [0.  , 0.46, 0.6 , 0.65, 0.69, 0.  , 0.83, 0.  , 0.59],
       [1.  , 0.  , 0.72, 0.69, 0.56, 0.  , 0.  , 0.  , 0.48],
       [0.  , 0.56, 0.66, 0.  , 0.68, 0.  , 0.68, 0.6 , 0.  ],
       [0.71, 0.  , 0.  , 0.  , 1.  , 0.41, 0.87, 0.  , 0.63],
       [0.56, 0.  , 0.75, 0.  , 0.4 , 0.79, 0.81, 0.8 , 0.  ],
       [0.  , 0.57, 0.39, 0.84, 0.  , 1.  , 0.  , 0.63, 0.  ],
       [0.  , 0.47, 0.49, 0.  , 0.  , 0.78, 0.56, 0.  , 0.44]],
      dtype=float32)

In [2]:
model = keras.models.load_model('sudoku.model')

In [3]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_28 (Conv2D)           (None, 9, 9, 64)          640       
_________________________________________________________________
batch_normalization_19 (Batc (None, 9, 9, 64)          256       
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 9, 9, 64)          36928     
_________________________________________________________________
batch_normalization_20 (Batc (None, 9, 9, 64)          256       
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 9, 9, 128)         8320      
_________________________________________________________________
flatten_9 (Flatten)          (None, 10368)             0         
_________________________________________________________________
dense_8 (Dense)              (None, 729)             